In [91]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Merge the databases that have information about RNA interactions

## Imports

In [92]:
import os
import pandas as pd
import re


In [93]:
fn_RNAInter = os.path.join('..', 'data', 'sRNA', 'RNAInter', 'Download_data_RR.csv')
fn_srnatarbase = os.path.join('..', 'data', 'sRNA', 'sRNATarBase', 'sRNATarBase.csv')
fn_merged = os.path.join('..', 'data', 'sRNA', 'merged_EcoCyc_RNAInter_sRNATarBase.csv')
data_rnainter = pd.read_csv(fn_RNAInter)
data_srnatarbase = pd.read_csv(fn_srnatarbase)
data_merged = pd.read_csv(fn_merged)
try:
    data_rnainter = data_rnainter.drop(columns=['Unnamed: 0', 'level_0'])
    data_srnatarbase = data_srnatarbase.drop(columns=['Unnamed: 0'])
    data_merged = data_merged.drop(columns=['Unnamed: 0'])
except:
    pass


In [94]:
print('Overlapping:', len(set([s for s in data_srnatarbase['sRNA'] if (s in data_rnainter[data_rnainter['Category1'] == 'sRNA']
      ['Interactor1.Symbol'].unique()) or (s in data_rnainter[data_rnainter['Category2'] == 'sRNA']['Interactor2.Symbol'].unique())])))
print('Non-overlapping:', len(set([s for s in data_srnatarbase['sRNA'] if (
    s not in data_rnainter[data_rnainter['Category1'] == 'sRNA']['Interactor1.Symbol'].unique()) and (s not in data_rnainter[data_rnainter['Category1'] == 'sRNA']['Interactor2.Symbol'].unique())])))

print('Overlapping:', len(set([s for s in data_rnainter[data_rnainter['Category1'] == 'sRNA']['Interactor1.Symbol'] if (
    s in data_srnatarbase['sRNA'].unique())])) +
    len(set([s for s in data_rnainter[data_rnainter['Category2'] == 'sRNA']['Interactor2.Symbol'] if (
        s not in data_srnatarbase['sRNA'].unique())])))

Overlapping: 36
Non-overlapping: 30
Overlapping: 23


In [95]:
data_rnainter.head()

,index,RNAInterID,Interactor1.Symbol,Category1,Species1,Interactor2.Symbol,Category2,Species2,Raw_ID1,Raw_ID2,score,strong,weak,predict,Sequence1,Sequence2
0,1029724,RR05384747,acnA,mRNA,Escherichia coli str. K-12 substr. MG1655,ryhB,sRNA,Escherichia coli str. K-12 substr. MG1655,NCBI:946724,NCBI:2847761,0.2292,Northern blot//Reporter assay,NaN,NaN,ATTCGGAACGAGGCCTGAAGCAGTGTCGCCGTCCCTCTGCCTTGCA...,TTTGAGAGCGTCTCTGTCCCTCGTTTTGCGGTTAAGCCGCATCCAT...
1,1029725,RR05384845,acrZ,mRNA,Escherichia coli str. K-12 substr. MG1655,omrB,sRNA,Escherichia coli str. K-12 substr. MG1655,NCBI:945365,NCBI:2847747,0.1778,Northern blot,Microarray,NaN,ACTTACTACTGTCTTCGGGGGGTCCGAGGTTTCTGGGGGGTCGTAC...,TGTTCTATACTTGGGTTCGACTTGGGTTAGACTTGTCTTTACTGTC...
2,1029732,RR05387056,arcZ,sRNA,Escherichia coli str. K-12 substr. MG1655,flhD,mRNA,Escherichia coli str. K-12 substr. MG1655,NCBI:2847690,NCBI:945442,0.2314,Northern blot//Reporter assay,NaN,NaN,CTCATGTTGACCGCTTGTTTAGCAGCTTCAAGGAAGCTGAAGGGCA...,CTCGTGCCGAATTCGGCACGAGCGATATTTCATCAGTTATCGGTAA...
3,1029733,RR05387057,arcZ,sRNA,Escherichia coli str. K-12 substr. MG1655,rpoS,mRNA,Escherichia coli str. K-12 substr. MG1655,NCBI:2847690,NCBI:947210,0.1996,Northern blot//RACE//RT-PCR//Beta-galactosidas...,NaN,NaN,CTCATGTTGACCGCTTGTTTAGCAGCTTCAAGGAAGCTGAAGGGCA...,ATCCTCGGGTCTTGCAGGCCACACAGGACACCCTGAACCGTCATGG...
4,1029734,RR05387126,argR,mRNA,Escherichia coli str. K-12 substr. MG1655,dsrA,sRNA,Escherichia coli str. K-12 substr. MG1655,NCBI:947861,NCBI:946470,0.2139,Primer extension assay,NaN,NaN,GACAATGGCGATAGTATGGCGGTTGTTTCTTTCCCATCTCTACTCA...,TGAGCAACTTTATTCACATAATTTCTACACCAAGAACTCGAGGTTA...


In [96]:
data_srnatarbase.head()

,Target,sRNA,Target Alias,sRNA Alias,Target ID,sRNA ID,Regulation,Target Type,sRNA Type,Target Strand,sRNA Strand,Target Binding Position,sRNA Binding Position,Target Genome Position,sRNA Genome Position,Target Sequence,sRNA Sequence
0,hns,dsrA,B1; bglY; cur; drc; drdX; drs; ECK1232; fimG; ...,ECK1952; IS095; JWR0036,"chromosome:NC_000913.3, Gene ID:945829","chromosome:NC_000913.3, Gene ID:946470",Repression,mRNA,trans-encoded antisense RNA,reverse,reverse,"['7..19', 'NA', '7..19; 401..411']","['31..43', 'NA', '31..54']",1292509..1292922,2025227..2025313,ATGAGCGAAGCACTTAAAATTCTGAACAACATCCGTACTCTTCGTG...,AACACATCAGATTTCCTGGTGTAACGAATTTTTTAAGTGCTTCTTG...
1,rbsD,dsrA,ECK3742; JW5857; rbsP,ECK1952; IS095; JWR0036,"chromosome:NC_000913.3, Gene ID:948267","chromosome:NC_000913.3, Gene ID:946470",Repression,mRNA,trans-encoded antisense RNA,forward,reverse,['NA'],['NA'],3933351..3933770,2025227..2025313,ATGAAAAAAGGCACCGTTCTTAATTCTGATATTTCATCGGTGATCT...,AACACATCAGATTTCCTGGTGTAACGAATTTTTTAAGTGCTTCTTG...
2,argR,dsrA,ECK3226; JW3206; Rarg; xerA,ECK1952; IS095; JWR0036,"chromosome:NC_000913.3, Gene ID:947861","chromosome:NC_000913.3, Gene ID:946470",Repression,mRNA,trans-encoded antisense RNA,forward,reverse,['NA'],['NA'],3384703..3385173,2025227..2025313,ATGCGAAGCTCGGCTAAGCAAGAAGAACTAGTTAAAGCATTTAAAG...,AACACATCAGATTTCCTGGTGTAACGAATTTTTTAAGTGCTTCTTG...
3,ilvI,dsrA,ECK0079; JW0076,ECK1952; IS095; JWR0036,"chromosome:NC_000913.3, Gene ID:948793","chromosome:NC_000913.3, Gene ID:946470",Repression,mRNA,trans-encoded antisense RNA,forward,reverse,['NA'],['NA'],85630..87354,2025227..2025313,ATGGAGATGTTGTCTGGAGCCGAGATGGTCGTCCGATCGCTTATCG...,AACACATCAGATTTCCTGGTGTAACGAATTTTTTAAGTGCTTCTTG...
4,rpoS,dsrA,abrD; appR; csi2; dpeB; ECK2736; JW5437; katF;...,ECK1952; IS095; JWR0036,"chromosome:NC_000913.3, Gene ID:947210","chromosome:NC_000913.3, Gene ID:946470",Induction,mRNA,trans-encoded antisense RNA,reverse,reverse,"['-119..-97', 'NA', 'NA']","['10..32', 'NA', 'NA']",2866559..2867551,2025227..2025313,ATGAGTCAGAATACGCTGAAAGTTCATGATTTAAATGAAGATGCGG...,AACACATCAGATTTCCTGGTGTAACGAATTTTTTAAGTGCTTCTTG...


In [97]:
data_srnatarbase[data_srnatarbase['Target Sequence'].isna() | data_srnatarbase['sRNA Sequence'].isna()]

,Target,sRNA,Target Alias,sRNA Alias,Target ID,sRNA ID,Regulation,Target Type,sRNA Type,Target Strand,sRNA Strand,Target Binding Position,sRNA Binding Position,Target Genome Position,sRNA Genome Position,Target Sequence,sRNA Sequence


# Rename columns

In [98]:
cols = ['Interactor 1', 'Interactor 2', 'ID 1', 'ID 2', 'Category 1', 'Category 2', 'Regulation', 'Binding position 1',
        'Binding position 2', 'Sequence 1', 'Sequence 2', 'Genome position 1', 'Genome position 2', 'Strand 1', 'Strand 2', 'Source DB']
cols_srnatarbase = {
    'sRNA': 'Interactor 1',
    'Target': 'Interactor 2',
    'sRNA ID': 'ID 1',
    'Target ID': 'ID 2',
    'sRNA Type': 'Category 1',
    'Target Type': 'Category 2',
    'sRNA Genome Position': 'Genome position 1',
    'Target Genome Position': 'Genome position 2',
    'sRNA Strand': 'Strand 1',
    'Target Strand': 'Strand 2',
    'sRNA Binding Position': 'Binding position 1',
    'Target Binding Position': 'Binding position 2',
    'sRNA Sequence': 'Sequence 1',
    'Target Sequence': 'Sequence 2'
}
cols_rnainter = {
    'Interactor1.Symbol': 'Interactor 1',
    'Interactor2.Symbol': 'Interactor 2',
    'Raw_ID1': 'ID 1',
    'Raw_ID2': 'ID 2',
    'Category1': 'Category 1',
    'Category2': 'Category 2',
    'Sequence1': 'Sequence 1',
    'Sequence2': 'Sequence 2'
}
data_rnainter = data_rnainter.rename(columns=cols_rnainter)
data_srnatarbase = data_srnatarbase.rename(columns=cols_srnatarbase)
data_srnatarbase.loc[data_srnatarbase['Category 1'].isna(),
                     'Category 1'] = 'sRNA'

data_rnainter['Regulation'] = ''
data_rnainter['Genome position 1'] = ''
data_rnainter['Genome position 2'] = ''
data_rnainter['Strand 1'] = ''
data_rnainter['Strand 2'] = ''
data_rnainter['Binding position 1'] = ''
data_rnainter['Binding position 2'] = ''
data_rnainter['Source DB'] = 'RNAInter'

data_srnatarbase['Source DB'] = 'sRNATarBase'

In [99]:
data = pd.concat([data_rnainter[cols], data_srnatarbase[cols]], axis=0)
data = data.reset_index(drop=True)
data

,Interactor 1,Interactor 2,ID 1,ID 2,Category 1,Category 2,Regulation,Binding position 1,Binding position 2,Sequence 1,Sequence 2,Genome position 1,Genome position 2,Strand 1,Strand 2,Source DB
0,acnA,ryhB,NCBI:946724,NCBI:2847761,mRNA,sRNA,,,,ATTCGGAACGAGGCCTGAAGCAGTGTCGCCGTCCCTCTGCCTTGCA...,TTTGAGAGCGTCTCTGTCCCTCGTTTTGCGGTTAAGCCGCATCCAT...,,,,,RNAInter
1,acrZ,omrB,NCBI:945365,NCBI:2847747,mRNA,sRNA,,,,ACTTACTACTGTCTTCGGGGGGTCCGAGGTTTCTGGGGGGTCGTAC...,TGTTCTATACTTGGGTTCGACTTGGGTTAGACTTGTCTTTACTGTC...,,,,,RNAInter
2,arcZ,flhD,NCBI:2847690,NCBI:945442,sRNA,mRNA,,,,CTCATGTTGACCGCTTGTTTAGCAGCTTCAAGGAAGCTGAAGGGCA...,CTCGTGCCGAATTCGGCACGAGCGATATTTCATCAGTTATCGGTAA...,,,,,RNAInter
3,arcZ,rpoS,NCBI:2847690,NCBI:947210,sRNA,mRNA,,,,CTCATGTTGACCGCTTGTTTAGCAGCTTCAAGGAAGCTGAAGGGCA...,ATCCTCGGGTCTTGCAGGCCACACAGGACACCCTGAACCGTCATGG...,,,,,RNAInter
4,argR,dsrA,NCBI:947861,NCBI:946470,mRNA,sRNA,,,,GACAATGGCGATAGTATGGCGGTTGTTTCTTTCCCATCTCTACTCA...,TGAGCAACTTTATTCACATAATTTCTACACCAAGAACTCGAGGTTA...,,,,,RNAInter
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
758,ryhB,msrA,"chromosome:NC_000913.3, Gene ID:2847761","chromosome:NC_000913.3, Gene ID:948734",sRNA,mRNA,No Interaction,['NA'],['NA'],GCGATCAGGAAGACCCTCGCGGAGAACCTGAAAGCACGACATTGCT...,ATGAGTTTATTTGATAAAAAGCATCTGGTTTCCCCCGCCGATGCCC...,3580927..3581016,4441538..4442176,reverse,reverse,sRNATarBase
759,gcvB,hdeA,"chromosome:NC_000913.3, Gene ID:2847720","chromosome:NC_000913.3, Gene ID:948025",trans-encoded antisense RNA,mRNA,Induction,['NA'],['NA'],ACTTCCTGAGCCGGAACGAAAAGTTTTATCGGAATGCGTGTTCTGG...,ATGAAAAAAGTATTAGGCGTTATTCTTGGTGGTCTGCTTCTTCTGC...,2942696..2942901,3656408..3656740,forward,reverse,sRNATarBase
760,gcvB,hdeB,"chromosome:NC_000913.3, Gene ID:2847720","chromosome:NC_000913.3, Gene ID:948026",trans-encoded antisense RNA,mRNA,Induction,['NA'],['NA'],ACTTCCTGAGCCGGAACGAAAAGTTTTATCGGAATGCGTGTTCTGG...,ATGAATATTTCATCTCTCCGTAAAGCGTTTATTTTTATGGGCGCTG...,2942696..2942901,3655966..3656292,forward,reverse,sRNATarBase
761,Esr41,fliC,"chromosome:NC_002695.1, Gene ID:","chromosome:NC_002695.1, Gene ID:",trans-encoded antisense RNA,mRNA,Induction,['NA'],['NA'],GATGCTCTAGGCATCACATTTTCTCCATGGGGTATTCCCTCCGCCG...,ATGGCACAAGTCATTAATACCAACAGCCTCTCGCTGATCACTCAAA...,1422406..1422479,2624379..2626136,forward,reverse,sRNATarBase


In [100]:
data[(data['Interactor 1'] == 'chiX') | (data['Interactor 2'] == 'chiX')]


,Interactor 1,Interactor 2,ID 1,ID 2,Category 1,Category 2,Regulation,Binding position 1,Binding position 2,Sequence 1,Sequence 2,Genome position 1,Genome position 2,Strand 1,Strand 2,Source DB
9,chbC,chiX,NCBI:945982,NCBI:5061500,mRNA,sRNA,,,,NCTATCATACTTTAGAAAAGCCATAGAGGGAAATTTTGTGAATAAG...,TGGATTGAACCNTCTTACACCCTGCTACACGCATCCTTAAGTCACC...,,,,,RNAInter
11,chiP,chiX,NCBI:945296,NCBI:5061500,mRNA,sRNA,,,,GCCTGTGACGGGGTCGGTTTGTACACCGATAAACTCAT,TGGATTGAACCNTCTTACACCCTGCTACACGCATCCTTAAGTCACC...,,,,,RNAInter
12,chiX,citA,NCBI:5061500,NCBI:945233,sRNA,mRNA,,,,TGGATTGAACCNTCTTACACCCTGCTACACGCATCCTTAAGTCACC...,GCCTGTGACCGACGTACTGGGGGATACGCAGATAAACTCAT,,,,,RNAInter
13,chiX,rpoS,NCBI:5061500,NCBI:947210,sRNA,mRNA,,,,TGGATTGAACCNTCTTACACCCTGCTACACGCATCCTTAAGTCACC...,ATCCTCGGGTCTTGCAGGCCACACAGGACACCCTGAACCGTCATGG...,,,,,RNAInter
594,chiX,rpoS,"chromosome:NC_000913.3, Gene ID:5061500","chromosome:NC_000913.3, Gene ID:947210",sRNA,mRNA,No Interaction,['NA'],['NA'],ACACCGTCGCTTAAAGTGACGGCATAATAATAAAAAAATGAAATTC...,ATGAGTCAGAATACGCTGAAAGTTCATGATTTAAATGAAGATGCGG...,507204..507287,2866559..2867551,forward,reverse,sRNATarBase
667,chiX,citA,"chromosome:NC_000913.3, Gene ID:5061500","chromosome:NC_000913.3, Gene ID:945233",trans-encoded antisense RNA,mRNA,Repression,['46..57'],['-37..-26'],ACACCGTCGCTTAAAGTGACGGCATAATAATAAAAAAATGAAATTC...,ATGTTGCAGCTTAACGAGAATAAACAGTTTGCATTTTTCCAAAGAC...,507204..507287,652235..653893,forward,forward,sRNATarBase
688,chiX,chiP,"chromosome:NC_000913.3, Gene ID:5061500","chromosome:NC_000913.3, Gene ID:945296",trans-encoded antisense RNA,mRNA,Repression,['81..92'],['45..56'],ACACCGTCGCTTAAAGTGACGGCATAATAATAAAAAAATGAAATTC...,ATGCGTACGTTTAGTGGCAAACGTAGTACGCTGGCGCTGGCTATCG...,507204..507287,708334..709740,forward,forward,sRNATarBase
722,chiX,chbC,"chromosome:NC_000913.3, Gene ID:5061500","chromosome:NC_000913.3, Gene ID:945982",trans-encoded antisense RNA,mRNA,Repression,['NA'],['NA'],ACACCGTCGCTTAAAGTGACGGCATAATAATAAAAAAATGAAATTC...,ATGAGTAATGTTATTGCATCGCTTGAAAAGGTACTCCTCCCTTTTG...,507204..507287,1819856..1821214,forward,reverse,sRNATarBase


In [101]:
data[data['Sequence 1'].isna() | data['Sequence 2'].isna()]

,Interactor 1,Interactor 2,ID 1,ID 2,Category 1,Category 2,Regulation,Binding position 1,Binding position 2,Sequence 1,Sequence 2,Genome position 1,Genome position 2,Strand 1,Strand 2,Source DB
181,mdtF,oxyS,NCBI:948030,NCBI:2847701,mRNA,sRNA,,,,NaN,TTTGCGAACTTATCGGTTTCGTATTGAGGGCAATAAACTCTCGAGG...,,,,,RNAInter
258,omrA,rrrD,NCBI:2847746,NCBI:947539,sRNA,mRNA,,,,CGCTCGAATCTGAGCCCCTACTCTCGTTACCATGTAACGGCGTACT...,NaN,,,,,RNAInter


## Sort interactions alphabetically to get rid of duplicates

In [102]:
cols = ['Interactor', 'ID', 'Category', 'Binding position', 'Sequence']

for interactors in data[['Interactor 1', 'Interactor 2']].iterrows():
    
    i1, i2 = interactors[1]
    
    cat1, cat2 = data.iloc[interactors[0]][['Category 1', 'Category 2']].to_list()
    sorted_interactors = [i1, i2] if cat2 == 'mRNA' else [i2, i1]

    if cat2 != 'mRNA':
        
        that_i = 1
        this_i = 2
        
        old = {}
        old[this_i] = data[[c + ' ' + str(this_i) for c in cols]].iloc[interactors[0]].to_list()
        old[that_i] = data[[c + ' ' + str(that_i) for c in cols]].iloc[interactors[0]].to_list()

        for this_i, that_i in [(1, 2), (2, 1)]:
            if (len(data[(data[f'Interactor {this_i}'] == sorted_interactors[that_i - 1]) & (
                        data[f'Interactor {that_i}'] == sorted_interactors[this_i - 1])][[
                  c + ' ' + str(that_i) for c in cols]])) > 1:
                print(i1, i2)
            data.loc[interactors[0], [c + ' ' + str(this_i) for c in cols]] = old[that_i]

omrA yzgL
omrB ygaY
ryhB yagJ


# Correct for RNAInter (false) sequences

In [103]:

# for i, s in [('Interactor 1', 'Sequence 1'), ('Interactor 2', 'Sequence 2')]:
#     for rna, seq in data[[i, s]].value_counts().index.tolist():
        
#         new_seq = seq
        
#         if (rna in data_srnatarbase['Interactor 1'].to_list()):
#             if data_srnatarbase[data_srnatarbase['Interactor 1'] == rna]['Sequence 1'].iloc[0] != seq:
#                 new_seq = data_srnatarbase[data_srnatarbase['Interactor 1'] == rna]['Sequence 1'].iloc[0]
                
#         elif (rna in data_srnatarbase['Interactor 2'].to_list()):
#             if data_srnatarbase[data_srnatarbase['Interactor 2'] == rna]['Sequence 2'].iloc[0] != seq:
#                 new_seq = data_srnatarbase[data_srnatarbase['Interactor 2'] == rna]['Sequence 2'].iloc[0] 
            
#         elif (rna in data_merged['Symbol'].to_list()):
#             if (data_merged[data_merged['Symbol'] == rna]['Sequence'].iloc[0] != seq):
#                 new_seq = data_merged[data_merged['Symbol'] == rna]['Sequence'].iloc[0]
#         else: 
#             print('Could not correct the sequence for', rna)
            
#         if type(new_seq) == str:
#             data.loc[data[i] == rna, s] = new_seq
        
            

In [104]:
data[data['Sequence 1'].isna()]

,Interactor 1,Interactor 2,ID 1,ID 2,Category 1,Category 2,Regulation,Binding position 1,Binding position 2,Sequence 1,Sequence 2,Genome position 1,Genome position 2,Strand 1,Strand 2,Source DB


# Drop RNAs that have previously been filtered

In [111]:
data = data[data['Interactor 1'].isin(data_merged['Name']) & data['Interactor 2'].isin(data_merged['Name'])]

# Drop duplicates

In [112]:
sum(data[['Interactor 1', 'Interactor 2']].value_counts() > 1)

0

In [113]:
data[['Interactor 1', 'Interactor 2']].value_counts()

Interactor 1  Interactor 2
AfaR          afaD            1
oxyS          yobF            1
rprA          ompF            1
              ompC            1
              ompA            1
                             ..
micF          phoE            1
              ompF            1
              ompC            1
              ompA            1
yzgL          omrA            1
Name: count, Length: 378, dtype: int64

In [114]:
for d in data[['Interactor 1', 'Interactor 2']].value_counts()[data[['Interactor 1', 'Interactor 2']].value_counts() > 1].index:
    
    data = data.drop(index=data[(data['Interactor 1'] == d[0]) & (data['Interactor 2'] == d[1]) & (data['Regulation'] == '')].index[0])
    
# data[data['Symbol'] == d].index[0]    
    

In [115]:
data[(data['Interactor 1'] == 'chiX') | (data['Interactor 2'] == 'chiX')]


,Interactor 1,Interactor 2,ID 1,ID 2,Category 1,Category 2,Regulation,Binding position 1,Binding position 2,Sequence 1,Sequence 2,Genome position 1,Genome position 2,Strand 1,Strand 2,Source DB
594,chiX,rpoS,"chromosome:NC_000913.3, Gene ID:5061500","chromosome:NC_000913.3, Gene ID:947210",sRNA,mRNA,No Interaction,['NA'],['NA'],ACACCGTCGCTTAAAGTGACGGCATAATAATAAAAAAATGAAATTC...,ATGAGTCAGAATACGCTGAAAGTTCATGATTTAAATGAAGATGCGG...,507204..507287,2866559..2867551,forward,reverse,sRNATarBase
667,chiX,citA,"chromosome:NC_000913.3, Gene ID:5061500","chromosome:NC_000913.3, Gene ID:945233",trans-encoded antisense RNA,mRNA,Repression,['46..57'],['-37..-26'],ACACCGTCGCTTAAAGTGACGGCATAATAATAAAAAAATGAAATTC...,ATGTTGCAGCTTAACGAGAATAAACAGTTTGCATTTTTCCAAAGAC...,507204..507287,652235..653893,forward,forward,sRNATarBase
688,chiX,chiP,"chromosome:NC_000913.3, Gene ID:5061500","chromosome:NC_000913.3, Gene ID:945296",trans-encoded antisense RNA,mRNA,Repression,['81..92'],['45..56'],ACACCGTCGCTTAAAGTGACGGCATAATAATAAAAAAATGAAATTC...,ATGCGTACGTTTAGTGGCAAACGTAGTACGCTGGCGCTGGCTATCG...,507204..507287,708334..709740,forward,forward,sRNATarBase
722,chiX,chbC,"chromosome:NC_000913.3, Gene ID:5061500","chromosome:NC_000913.3, Gene ID:945982",trans-encoded antisense RNA,mRNA,Repression,['NA'],['NA'],ACACCGTCGCTTAAAGTGACGGCATAATAATAAAAAAATGAAATTC...,ATGAGTAATGTTATTGCATCGCTTGAAAAGGTACTCCTCCCTTTTG...,507204..507287,1819856..1821214,forward,reverse,sRNATarBase


## Add fields from reference dataset

In [116]:
data['Sequence before 1'] = ''
data['Sequence before 2'] = ''
data['Sequence after 1'] = ''
data['Sequence after 2'] = ''
for i in [1, 2]:
    for n in data[f'Interactor {i}'].unique():
        # filt = data_merged['Alias'].str.lower().str.contains(n.lower())
        # data_merged[filt == True]
        seqs = data[data[f'Interactor {i}'] == n][f'Sequence {i}'].unique()
        for s in seqs:
            data.loc[(data[f'Interactor {i}'] == n) & (data[f'Sequence {i}'] == s), f'Sequence before {i}'] = data_merged[(data_merged['Name'] == n) & (data_merged['Sequence'] == s)]['Sequence before'].iloc[0]
            data.loc[(data[f'Interactor {i}'] == n) & (data[f'Sequence {i}'] == s), f'Sequence after {i}'] = data_merged[(data_merged['Name'] == n) & (data_merged['Sequence'] == s)]['Sequence after'].iloc[0]

In [119]:
data_merged[(data_merged['Name'] == n) & (data_merged['Sequence'] == s)]

,Name,ID,Category,Symbol,Sequence,Database,Alias,Genomic position,Strand,Sequence before,Sequence after
589,mreB,"chromosome:NC_000913.3, Gene ID:948588",mRNA,mreB,ATGTTGAAAAAATTTCGTGGCATGTTTTCCAATGACTTGTCCATTG...,sRNATarBase,ECK3239; envB; JW3220; mon; rodY,3400044..3401087,reverse,NaN,atactaagggataatcctgaaagctggggcggaaaagaaaatccgc...


In [118]:
data

,Interactor 1,Interactor 2,ID 1,ID 2,Category 1,Category 2,Regulation,Binding position 1,Binding position 2,Sequence 1,Sequence 2,Genome position 1,Genome position 2,Strand 1,Strand 2,Source DB,Sequence before 1,Sequence before 2,Sequence after 1,Sequence after 2
50,dsrA,dsrB,NCBI:946470,NCBI:946468,sRNA,mRNA,,,,TGAGCAACTTTATTCACATAATTTCTACACCAAGAACTCGAGGTTA...,TAAATGNAAACAAGTTTTATTTTTATCATCAGAACATTCTAATAAA...,,,,,RNAInter,ttcatcaccttatccgcaatttttttcgctgacaagaaaatattcg...,atcgttttcctcctgtggctttgtgccagtgtagaacaatttcgtt...,ttcatcaccttatccgcaatttttttcgctgacaagaaaatattcg...,atcgttttcctcctgtggctttgtgccagtgtagaacaatttcgtt...
53,dsrA,ilvH,NCBI:946470,NCBI:947267,sRNA,mRNA,,,,TGAGCAACTTTATTCACATAATTTCTACACCAAGAACTCGAGGTTA...,ATTCGCCACAACAAGGACAGGAAGGTTCGGCGCAAGGAGCCCAAGA...,,,,,RNAInter,ttcatcaccttatccgcaatttttttcgctgacaagaaaatattcg...,ATTCGCCACAACAAGGACAGGAAGGTTCGGCGCAAGGAGCCCAAGA...,ttcatcaccttatccgcaatttttttcgctgacaagaaaatattcg...,ATTCGCCACAACAAGGACAGGAAGGTTCGGCGCAAGGAGCCCAAGA...
74,oxyS,fhlA,NCBI:2847701,NCBI:947181,sRNA,mRNA,,,,TTTGCGAACTTATCGGTTTCGTATTGAGGGCAATAAACTCTCGAGG...,CACCAGATTGCCAAGCTGATGGTCGAGCTGTCCAAATCCCAGGATG...,,,,,RNAInter,TTTGCGAACTTATCGGTTTCGTATTGAGGGCAATAAACTCTCGAGG...,CACCAGATTGCCAAGCTGATGGTCGAGCTGTCCAAATCCCAGGATG...,TTTGCGAACTTATCGGTTTCGTATTGAGGGCAATAAACTCTCGAGG...,CACCAGATTGCCAAGCTGATGGTCGAGCTGTCCAAATCCCAGGATG...
138,sgrS,hfq,NCBI:4056038,NCBI:948689,sRNA,mRNA,,,,CCAGAACATTGGCTTTCAAAAGTTTTTTAATTGATGGAAACTCTTT...,ATGGCTAAGGGGCAATCTTTACAAGATCCGTTCCTGAACGCACTGC...,,,,,RNAInter,ttattctcgccgcgctaaaaagggaacgtatgatctggataatgac...,aaggtttcgggctgtttttttacacggggagccagcgatcctgcgt...,ttattctcgccgcgctaaaaagggaacgtatgatctggataatgac...,aaggtttcgggctgtttttttacacggggagccagcgatcctgcgt...
230,oxyS,nusG,NCBI:2847701,NCBI:948485,sRNA,mRNA,,,,TTTGCGAACTTATCGGTTTCGTATTGAGGGCAATAAACTCTCGAGG...,ATGTCTGAAGCTCCTAAAAAGCGCTGGTACGTCGTTCAGGCGTTTT...,,,,,RNAInter,TTTGCGAACTTATCGGTTTCGTATTGAGGGCAATAAACTCTCGAGG...,aacccagcgatcaaaaaagcggcgatttaatcgttgcacaaggcgt...,TTTGCGAACTTATCGGTTTCGTATTGAGGGCAATAAACTCTCGAGG...,aacccagcgatcaaaaaagcggcgatttaatcgttgcacaaggcgt...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
758,ryhB,msrA,"chromosome:NC_000913.3, Gene ID:2847761","chromosome:NC_000913.3, Gene ID:948734",sRNA,mRNA,No Interaction,['NA'],['NA'],GCGATCAGGAAGACCCTCGCGGAGAACCTGAAAGCACGACATTGCT...,ATGAGTTTATTTGATAAAAAGCATCTGGTTTCCCCCGCCGATGCCC...,3580927..3581016,4441538..4442176,reverse,reverse,sRNATarBase,TTTGAGAGCGTCTCTGTCCCTCGTTTTGCGGTTAAGCCGCATCCAT...,atggtgtcgctctcccgaaaatcggtcattggggttaaggttgtaa...,TTTGAGAGCGTCTCTGTCCCTCGTTTTGCGGTTAAGCCGCATCCAT...,atggtgtcgctctcccgaaaatcggtcattggggttaaggttgtaa...
759,gcvB,hdeA,"chromosome:NC_000913.3, Gene ID:2847720","chromosome:NC_000913.3, Gene ID:948025",trans-encoded antisense RNA,mRNA,Induction,['NA'],['NA'],ACTTCCTGAGCCGGAACGAAAAGTTTTATCGGAATGCGTGTTCTGG...,ATGAAAAAAGTATTAGGCGTTATTCTTGGTGGTCTGCTTCTTCTGC...,2942696..2942901,3656408..3656740,forward,reverse,sRNATarBase,GGTAGAGCGCCATTGCATTCTTTAGCGCCTCGATCCCTTTATTTTT...,atcgtaatatcctcaactataaagtgaaagagccgtcacgaatcaa...,GGTAGAGCGCCATTGCATTCTTTAGCGCCTCGATCCCTTTATTTTT...,atcgtaatatcctcaactataaagtgaaagagccgtcacgaatcaa...
760,gcvB,hdeB,"chromosome:NC_000913.3, Gene ID:2847720","chromosome:NC_000913.3, Gene ID:948026",trans-encoded antisense RNA,mRNA,Induction,['NA'],['NA'],ACTTCCTGAGCCGGAACGAAAAGTTTTATCGGAATGCGTGTTCTGG...,ATGAATATTTCATCTCTCCGTAAAGCGTTTATTTTTATGGGCGCTG...,2942696..2942901,3655966..3656292,forward,reverse,sRNATarBase,GGTAGAGCGCCATTGCATTCTTTAGCGCCTCGATCCCTTTATTTTT...,attttgtaacccattcaatatagagattatatatgaacaggaggaa...,GGTAGAGCGCCATTGCATTCTTTAGCGCCTCGATCCCTTTATTTTT...,attttgtaacccattcaatatagagattatatatgaacaggaggaa...
761,Esr41,fliC,"chromosome:NC_002695.1, Gene ID:","chromosome:NC_002695.1, Gene ID:",trans-encoded antisense RNA,mRNA,Induction,['NA'],['NA'],GATGCTCTAGGCATCACATTTTCTCCATGGGGTATTCCCTCCGCCG...,ATGGCACAAGTCATTAATACCAACAGCCTCTCGCTGATCACTCAAA...,1422406..1422479,2624379..2626136,forward,reverse,sRNATarBase

### Fill in alias if none are found

In [21]:
data_merged.loc[data_merged['Alias'].apply(lambda x: type(x) == float), 'Alias'] = data_merged[data_merged['Alias'].apply(lambda x: type(x) == float)]['Symbol']

# Save

In [32]:
data[['Interactor 1', 'Interactor 2', 'ID 1', 'ID 2', 'Category 1',
      'Category 2', 'Regulation', 'Binding position 1', 'Binding position 2',
      'Sequence 1', 'Sequence 2', 'Source DB', 'Genome position 1', 'Genome position 2',
      'Strand 1', 'Strand 2', 'Sequence 1 before', 'Sequence 2 before', 'Sequence 1 after',
      'Sequence 2 after']].to_csv(os.path.join('..', 'data', 'sRNA', 'merged_inter.csv'))